In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/21 21:06:40 WARN Utils: Your hostname, DESKTOP-C98VC8U resolves to a loopback address: 127.0.1.1; using 172.22.196.85 instead (on interface eth0)
21/08/21 21:06:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/21 21:06:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/21 21:06:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/08/21 21:06:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/08/21 21:06:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/08/21 21:06:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
21/08/21 21

In [3]:
customers = spark.read.parquet("./output/customers.parquet")
customers.show(3)

+----------------+---------+--------+--------------------+--------+--------------+
|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+----------------+---------+--------+--------------------+--------+--------------+
|               1|    Terri| Leonard|          3124065463|   65369|          Mrs.|
|               2|  Maurice|    West|001-468-075-5301x308|   27061|           Dr.|
|               3|   Taylor|    Hall|    001-004-938-2651|   88302|           Mr.|
+----------------+---------+--------+--------------------+--------+--------------+
only showing top 3 rows



In [4]:
tickets = spark.read.parquet("./output/tickets.parquet")
tickets.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
only showing top 3 rows



In [5]:
#broadcast joining because one table is very small, reduces shuffling
import pyspark.sql.functions as F
join_condition = customers.CustomerIdentity == tickets.customer_id
combined_df =  tickets.join(F.broadcast(customers), join_condition).drop("CustomerIdentity")
combined_df.count()

1000

In [6]:
combined_df.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Net_sales: double (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_season: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LASTNAME: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- Customer_Title: string (nullable = true)



In [7]:
combined_df.select("Date").where(F.col("Date").isNull()).count()

0

In [8]:
combined_df.show(5)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+--------------------+--------+--------------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+--------------------+--------+--------------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|   Justin| Higgins|          1889149702|   67579|          Mrs.|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05| Jennifer|  Dawson|    479.687.6222x409|   23

In [11]:
import pyspark.sql.functions as F
# 3. For each Customer, a list of Events
Edf = combined_df.groupBy("customer_id")\
                 .agg(F.collect_list(F.col("event_code"))\
                 .alias("Events_list"))
Edf.show(5, truncate=False)

+-----------+---------------------------------------------------------------------------------+
|customer_id|Events_list                                                                      |
+-----------+---------------------------------------------------------------------------------+
|199        |[CHL-ARS, CHL-ARS, CHL-ARS, CHL-ARS, CHL-ARS, CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC]|
|198        |[CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC, CHL-ARS]                                    |
|197        |[CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC, BTH-SRC, CHL-ARS]                           |
|196        |[F1-SILV, BTH-SRC, BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS]                           |
|195        |[BTH-SRC, BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS, BTH-SRC, CHL-ARS, F1-SILV]         |
+-----------+---------------------------------------------------------------------------------+
only showing top 5 rows



In [13]:
# combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()
Tots = combined_df.groupBy("customer_id").agg(F.count("event_name").alias("Totals"))
Tots.show(2)

+-----------+------+
|customer_id|Totals|
+-----------+------+
|        148|     5|
|         31|    10|
+-----------+------+
only showing top 2 rows



In [11]:
MultiEvent = Tots.withColumn("MultiEvent", F.col("Totals") > 1)
MultiEvent.show()

+-----------+------+----------+
|customer_id|Totals|MultiEvent|
+-----------+------+----------+
|        148|     5|      true|
|         31|    10|      true|
|        137|     7|      true|
|         85|     3|      true|
|         65|     3|      true|
|         53|     1|     false|
|        133|     9|      true|
|         78|     3|      true|
|        108|    11|      true|
|        155|     4|      true|
|         34|     8|      true|
|        193|     6|      true|
|        115|     4|      true|
|        101|     5|      true|
|        126|     1|     false|
|         81|     2|      true|
|        183|     9|      true|
|         28|     1|     false|
|         76|    10|      true|
|         26|     4|      true|
+-----------+------+----------+
only showing top 20 rows



In [ ]:
MultiEvent1 = Tots.withColumn("MultiEvent", F.col("Totals") > 1)
MultiEvent.show()

In [14]:
def multiEvent(x):
    
   if x > 1:

        return   "True"
   else:
        return " "
    

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
multiEvent_udf = udf(multiEvent, StringType())

#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
Tots.withColumn("MultiEvent", multiEvent_udf(F.col("Totals"))).show(5)

In [17]:
cols = ["customer_id", "event_name", "event_season", "Date", "rank"]
combined_df.groupBy("customer_id").agg(F.max("quantity")).show()

+-----------+-------------+
|customer_id|max(quantity)|
+-----------+-------------+
|        148|            4|
|         31|            5|
|        137|            5|
|         85|            5|
|         65|            4|
|         53|            2|
|        133|            5|
|         78|            4|
|        108|            5|
|        155|            5|
|         34|            4|
|        193|            5|
|        115|            5|
|        101|            5|
|        126|            5|
|         81|            3|
|        183|            5|
|         28|            3|
|         76|            5|
|         26|            5|
+-----------+-------------+
only showing top 20 rows



In [29]:
# 5. Largest Order by Quantity for each Customer
from pyspark.sql.window import Window
cols = ["customer_id", "quantity"]
windowSpec1 = Window.partitionBy(["customer_id"])\
                    .orderBy(F.desc("quantity"))

combined_df.withColumn("rank", F.rank().over(windowSpec1)).filter("rank == 1").select(cols).show()

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|        148|       4|
|         31|       5|
|         31|       5|
|         31|       5|
|         31|       5|
|         85|       5|
|        137|       5|
|        137|       5|
|         65|       4|
|         53|       2|
|        133|       5|
|        133|       5|
|        133|       5|
|         78|       4|
|         78|       4|
|        108|       5|
|        108|       5|
|        108|       5|
|        155|       5|
|         34|       4|
+-----------+--------+
only showing top 20 rows



In [79]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id").orderBy(F.desc("quantity"))

F = combined_df.withColumn("dddd", F.collect_list(F.col("quantity")).over(WindowSpec)).select("customer_id", "dddd")
F.show()

+-----------+--------------------+
|customer_id|                dddd|
+-----------+--------------------+
|        148|                 [4]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|     [4, 3, 3, 3, 2]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         85|                 [5]|
|         85|              [5, 4]|
|         85|           [5, 4, 1]|
|        137|              [5, 5]|
|        137|              [5, 5]|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
F.collect()[2]["dddd"]

In [59]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id")

df22 = combined_df.withColumn("EventsList", 
                              F.collect_list(F.col("event_code"))
                              .over(WindowSpec)
                              )\
.select("customer_id", "EventsList")
df22.show(5, truncate=False)



+-----------+---------------------------------------------+
|customer_id|EventsList                                   |
+-----------+---------------------------------------------+
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
+-----------+---------------------------------------------+
only showing top 5 rows



In [69]:
combined_df.groupBy("customer_id").agg(F.max("quantity"))

WindowSpec = Window.partitionBy("customer_id")
WindowSpec.orderBy(F.col("quantity"))

# .select(col("*"), rank().over(win).as("rank"))
# .filter(col("rank") === 2)
# .show()

AttributeError: 'WindowSpec' object has no attribute 'show'

In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="event_code", outputCol="IndexEvents")
indexed = indexer.fit(combined_df).transform(combined_df)
indexed.show(10)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|IndexEvents|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|        0.0|
|        2|     376|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|              46|   Justin| Higgins|          1889149702|   67579|          Mrs.|        0.0|
|        3|     282|       5|     47.0|   CHL-ARS|Chelsea vs Arse

In [18]:
combined_df.groupBy("Customer_Title").agg(F.sum("quantity").alias("Totals")).sort("Totals", ascending=False).show()

+--------------+------+
|Customer_Title|Totals|
+--------------+------+
|           Dr.|  1023|
|           Mr.|   996|
|          Mrs.|   691|
|          Miss|   131|
|           Ms.|   128|
+--------------+------+



In [20]:
combined_df.groupBy("customer_id").agg(F.max("quantity")).show(5)

+-----------+-------------+
|customer_id|max(quantity)|
+-----------+-------------+
|        148|            4|
|         31|            5|
|        137|            5|
|         85|            5|
|         65|            4|
+-----------+-------------+
only showing top 5 rows



In [22]:
#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
combined_df.groupBy("customer_id").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()

+-----------+------+
|customer_id|Totals|
+-----------+------+
|         53|     1|
|         86|     1|
|        121|     1|
|        126|     1|
|        175|     1|
|         28|     1|
|        149|     1|
|        145|     2|
|         15|     2|
|        156|     2|
|          5|     2|
|         61|     2|
|         24|     2|
|         62|     2|
|          1|     2|
|        117|     2|
|        171|     2|
|         13|     2|
|         81|     2|
|         91|     2|
+-----------+------+
only showing top 20 rows



In [24]:
combined_df.select("customer_id").distinct().count()

198